In [7]:
# Imports
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [8]:
# Função para calcular a porcentagem de diferença
def calculate_difference_percentage(img1, img2):
    # Converter as imagens para escala de cinza
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    # Calcular a diferença absoluta
    diff = cv2.absdiff(img1_gray, img2_gray)
    
    # Aplicar um limiar para destacar as diferenças
    _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    
    # Calcular a porcentagem de pixels diferentes
    non_zero_count = np.count_nonzero(thresh)
    total_pixels = thresh.size
    percentage_difference = (non_zero_count / total_pixels) * 100
    
    return percentage_difference


In [9]:

# Função para mostrar imagens lado a lado
def plot_images_with_difference(img1, img2, diff, current_frame):
    fig, axes = plt.subplots(1, 4, figsize=(20, 15))

    # Mostrar imagem 1
    axes[0].imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Imagem 1')
    axes[0].axis('off')

    # Mostrar imagem 2
    axes[1].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
    axes[1].set_title('Imagem 2')
    axes[1].axis('off')

    # Mostrar diferença
    axes[2].imshow(diff, cmap='gray')
    axes[2].set_title('Diferença')
    axes[2].axis('off')

    # Mostrar imagem 1 x imagem 2
    combined = cv2.bitwise_and(img1, img2)
    axes[3].imshow(cv2.cvtColor(combined, cv2.COLOR_BGR2RGB))
    axes[3].set_title('Imagem 1 & Imagem 2')
    axes[3].axis('off')

    # Ajustar o layout
    plt.tight_layout()
    plt.show()


In [10]:
def extract_date_from_path(path):
    # Extrair a data do caminho do arquivo usando regex
    match = re.search(r'\d{2}-\d{2}-\d{2}', path)
    return match.group(0) if match else 'unknown_date'

In [11]:
# Função para salvar a imagem
def save_frame(frame, frame_number, output_dir, video_index, original_name, date):
    # Padronizar o nome do arquivo
    file_name = f"{date}_video{video_index}_{original_name}_{frame_number}.png"
    os.makedirs(output_dir + f'/video_{video_index}', exist_ok=True)
    file_path = os.path.join(output_dir + f'/video_{video_index}', file_name)
    cv2.imwrite(file_path, frame)


In [12]:

def process_videos(video_paths):
    last_date = None
    video_counter = 0

    for i, video_path in enumerate(video_paths):
        date = extract_date_from_path(video_path)

        # Verifica se a data mudou e reinicia o contador
        if date != last_date:
            video_counter = 1
            last_date = date
        else:
            video_counter += 1

        cap = cv2.VideoCapture(video_path)
        original_name = os.path.splitext(os.path.basename(video_path))[0]

        # Pular para o frame inicial
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

        # Lista para armazenar os frames com diferenças significativas
        frames_list = []

        # Diretório para salvar as imagens
        output_dir = '../dataset/'
        os.makedirs(output_dir, exist_ok=True)

        # Ler o primeiro frame do período especificado
        ret, prev_frame = cap.read()
        if not ret:
            print(f"Não foi possível ler o vídeo {video_path}.")
            cap.release()
            continue

        while True:
            current_pos = cap.get(cv2.CAP_PROP_POS_FRAMES)
            
            # Pular frames
            cap.set(cv2.CAP_PROP_POS_FRAMES, current_pos + 20)

            ret, frame = cap.read()
            if not ret:
                break

            # Adicionar o frame à lista
            frames_list.append((int(current_pos), frame))

        # Loop para separar a lista em 5 partes e salvar arquivos em outputs diferentes
        num_frames = len(frames_list)
        portion_size = num_frames // 5  # 20% da lista

        for j in range(5):
            # Diretório correspondente para cada pessoa
            person_name = ["giuliano", "guilherme", "marcelo", "mariana", "samuel"][j]
            person_output_dir = os.path.join(output_dir + date, person_name)
            os.makedirs(person_output_dir, exist_ok=True)

            # Definir o intervalo de frames para cada pessoa
            start_index = j * portion_size
            if j == 4:  # Garantir que o último grupo receba todos os frames restantes
                end_index = num_frames
            else:
                end_index = (j + 1) * portion_size

            # Salvar os frames do intervalo correspondente
            for k, (frame_number, frame) in enumerate(frames_list[start_index:end_index]):
                save_frame(frame, frame_number, person_output_dir, video_counter, original_name, date)

        # Liberar recursos
        cap.release()

# Lista de caminhos dos vídeos
video_paths = [
    '../videos/29-07-24/00000000196000400-edit.mp4',
    '../videos/29-07-24/00000000196000500-edit.mp4',
    '../videos/06-08-24/imagem_termica_edit.mp4',
    '../videos/06-08-24/imagem_termica2-edit.mp4'
]

# Processar os vídeos
process_videos(video_paths)